# A9 Transmission grating EXP2
by H cyan 
 
huangzw29@mail2.sysu.edu.cn

2021.3.24

## EXP2 Measure the wavelengh of the two yellow spectrual lines in the second diffraction order of the diffraction spectrum of Hg

In [21]:
import math
import numpy as np
import pandas as pd

def dms2rad(dms):
    '''
    Convert DMS to radian.
    args:
        dms - [degrees,minutes,seconds]
    return:
        rad - value measured in deg of radian.
    '''
    deg=dms[0]+dms[1]/60+dms[2]/3600
    rad=math.radians(deg)
    return rad

# import data from exp1

d=3.424451355053853e-06 # grating constant
ud=1.8245960906790115e-09 # Sb of the grating constant
uphi=dms2rad([0,1,0])/math.sqrt(3) # uphi
sd=2.8340444255744426e-09 # the combined standard uncertainty S of the grating constant

### 2.1 import data
**Warning: measurments of phi1 may have some problems.**

In [22]:
reedAListdeg=[[94,32,0],[94,28,0],[74,45,0],[54,35,0],[54,31,0]]
reedBListdeg=[[274,31,0],[274,28,0],[254,40,0],[244,34,0],[234,28,0]]
reedAListrad=[]
reedBListrad=[]


for x in reedAListdeg:
    reedAListrad.append(dms2rad(x))
for x in reedBListdeg:
    reedBListrad.append(dms2rad(x))


df = pd.DataFrame({'A':np.array(reedAListrad),\
     'B':np.array(reedBListrad)})
 
print(df)

          A         B
0  1.649918  4.791220
1  1.648754  4.790347
2  1.304634  4.444772
3  0.952659  4.268494
4  0.951495  4.092215


### 2.2 process data and caculate the list of phi

In [23]:
DFdiff=df.diff()
print(DFdiff)

aa=-(DFdiff.iloc[1]['A']+DFdiff.iloc[1]['B']+DFdiff.iloc[2]['A']+DFdiff.iloc[2]['B'])/2
bb=-(DFdiff.iloc[2]['A']+DFdiff.iloc[2]['B'])/2
cc=-(DFdiff.iloc[3]['A']+DFdiff.iloc[3]['B'])/2
dd=-(DFdiff.iloc[3]['A']+DFdiff.iloc[3]['B']+DFdiff.iloc[4]['A']+DFdiff.iloc[4]['B'])/2

phiList=[aa,bb,0,cc,dd]
print({'list of phi':phiList})

          A         B
0       NaN       NaN
1 -0.001164 -0.000873
2 -0.344121 -0.345575
3 -0.351975 -0.176278
4 -0.001164 -0.176278
{'list of phi': [0.3458660801035428, 0.3448479713732123, 0, 0.2641264934684755, 0.35284739711152047]}


### 2.3 Calculate the wavelengh lambda

In [24]:
phi1List=np.array([phiList[1],phiList[3]])
phi1=phi1List.mean()
phi2List=np.array([phiList[0],phiList[4]])
phi2=phi2List.mean()

lambda1=(d*math.sin(phi1))/2
lambda2=(d*math.sin(phi2))/2

print({'[phi1,phi2]':[phi1,phi2]}  )
print({'[lambda1,lambda2]':[lambda1,lambda2]})

{'[phi1,phi2]': [0.3044872324208439, 0.34935673860753164]}
{'[lambda1,lambda2]': [5.133321651721434e-07, 5.860836760401778e-07]}


### 2.4 Uncertainty & degree of freedom

In [25]:
def Slambda(phik,phikList):
    '''
    calculate the uncertainty of lambda
    args:
        phik - angle of the kth line
        phikList - list of measurements of angle of the kth line
    return:
        [Sak,Sbk,Slambdak]
    '''
    Sphik=phikList.std(ddof=1) 
    Sak=math.sqrt((math.sin(phik)*sd/2)**2+(d*math.cos(phik)*Sphik/2)**2)
    Sbk=math.sqrt((math.sin(phik)*ud/2)**2+(d*math.cos(phik)*uphi/2)**2)
    Slambdak=math.sqrt((Sak)**2+(Sbk)**2)
    return [Sphik,Sak,Sbk,Slambdak]

def Vlambda(phik,phikList):
    '''
    calculate the DOF of lambda
    fuc Slambda() is invoked.
    args:
        phik - angle of kth line
        phikList - the list of measurements of the angle of the kth line
    return:
        [Vak,Vbk,Vlambdak]
    '''
    Slist=Slambda(phik,phikList)
    Vak=Slist[1]**4/((math.sin(phik)*sd/2)**4+(d*math.cos(phik)*Slist[0]/2)**4)
    Vbk=Slist[2]**4/((math.sin(phik)*ud/2)**4+(d*math.cos(phik)*uphi/2)**4)
    Vlambdak=Slist[3]**4/((Slist[1]**4/Vak)+(Slist[2]**4/Vbk))
    return [Vak,Vbk,Vlambdak]


Slist1=dict(zip(['Sphi1','Sa1','Sb1','Slambda1'],Slambda(phi1,phi1List)))
Vlist1=dict(zip(['Va1','Vb1','Vlambda1'],Vlambda(phi1,phi1List)))
Slist2=dict(zip(['Sphi2','Sa2','Sb2','Slambda2'],Slambda(phi2,phi2List)))
Vlist2=dict(zip(['Va2','Vb2','Vlambda2'],Vlambda(phi2,phi2List)))
print(Slist1)
print(Slist2)
print(Vlist1)
print(Vlist2)



{'Sphi1': 0.05707870441383947, 'Sa1': 9.323701546741646e-08, 'Sb1': 3.873831260382876e-10, 'Slambda1': 9.323782021775066e-08}
{'Sphi2': 0.004936536597953982, 'Sa2': 7.95667511465472e-09, 'Sb2': 4.129364165898476e-10, 'Slambda2': 7.96738321937082e-09}
{'Va1': 1.000041523174276, 'Vb1': 1.9999820516795064, 'Vlambda1': 1.000076049834829}
{'Va2': 1.007459816520603, 'Vb2': 1.9595038717589106, 'Vlambda2': 1.0128903600346324}


### 2.5 Caculate angular dispersion D and S,V

In [26]:
D=(phi1-phi2)/(lambda1-lambda2)
print('D='+str(D))

Sad=math.sqrt((Slist1['Sphi1'])**2+(Slist2["Sphi2"])**2+(Slist1['Slambda1']*(D/(lambda1-lambda2)))**2+(Slist2["Slambda2"]*(D/(lambda1-lambda2)))**2)
Sbd=math.sqrt((uphi)**2+(uphi)**2+(Slist1['Sb1']*(D/(lambda1-lambda2)))**2+(Slist2['Sb2']*(D/(lambda1-lambda2)))**2)
Sd=math.sqrt((Sad)**2+(Sbd)**2)

Vad=Sad**4/((Slist1['Sphi1'])**4+(Slist2["Sphi2"])**4+((Slist1['Slambda1']*(D/(lambda1-lambda2)))**4)/Vlist1['Vlambda1']+((Slist2["Slambda2"]*(D/(lambda1-lambda2)))**4)/Vlist2['Vlambda2'])
Vbd=Sbd**4/((uphi)**4+(uphi)**4+((Slist1['Sb1']*(D/(lambda1-lambda2)))**4)/Vlist1['Vb1']+((Slist2['Sb2']*(D/(lambda1-lambda2)))**4)/Vlist2['Vb2'])
Vd=Sd**4/((Sad**4/Vad)+(Sbd**4/Vbd))

print({'[Sad,Sbd,Sd]':[Sad,Sbd,Sd]})
print({'[Vad,Vbd,Vd]':[Vad,Vbd,Vd]})


D=616750.1630045527
{'[Sad,Sbd,Sd]': [793303.2531719295, 4799.9524867340415, 793317.7743105479]}
{'[Vad,Vbd,Vd]': [1.0146812464775161, 3.937910475416706, 1.0147555417084402]}
